Idea of cutset conditioning: it's a way to run exact inference on a model with loops. You cut the loop by observing one of the variables in the loop to all the possible states, then fuse the results in a smart way.

 Cutset Conditioning is a technique for solving nearly-tree-structured CSPs in which some variables are assigned to separately from the rest, removed from the constraint graph, and leaving a tree-structured CSP for those remaining.

 Cutsets are some set of variables that are cut (severing edges) from the original constraint graph and solved separately.

 Conditioning is the process of assigning a value to some variable in a cutset, performing forward checking on its neighbor domains before cutting, and finally, severing it from the original graph.

https://forns.lmu.build/classes/spring-2019/cmsi-282/lecture-13M.html#backtracking++

In [19]:
import src.data.breathe_data as bd

# import src.inference.long_inf_slicing as slicing
import src.models.builders as mb

# import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
# import src.models.helpers as mh

# import pandas as pd
# import numpy as np

Figure per entry that has the AR from obs FEF2575 on top and on the bottom the point mass AR obtained by repeating model runs with several point mass HFEV1 (3, 3.5, 4, 4.5, 5, etc)

In [9]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx")

In [11]:
df.columns

Index(['ID', 'Date Recorded', 'FEV1', 'O2 Saturation', 'FEF2575', 'ecFEV1',
       'ecFEF2575', 'Sex', 'Height', 'Age', 'Predicted FEV1',
       'Healthy O2 Saturation', 'ecFEV1 % Predicted', 'FEV1 % Predicted',
       'O2 Saturation % Healthy', 'ecFEF2575%ecFEV1', 'idx ecFEV1 (L)',
       'idx ecFEF25-75 % ecFEV1 (%)', 'idx O2 saturation (%)'],
      dtype='object')

In [100]:
# With each run I should retrieve
# 1/ the message from FEF25-75%FEFV1 to AR
# 2/ the point mass message from the factor ecFEV1, HFEV1 to AR
# Use the point in time model, there is no shared variables.

df_for_ID = df[df.ID == '272']
df_for_ID.reset_index(inplace=True, drop=True)
height = df_for_ID.loc[0, "Height"]
age = df_for_ID.loc[0, "Age"]
sex = df_for_ID.loc[0, "Sex"]
(
    model,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    ecFEF2575prctecFEV1,
) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(height, age, sex)

FEV_to_AR_key ="['ecFEV1 (L)', 'Healthy FEV1 (L)', 'Airway resistance (%)'] -> Airway resistance (%)"
FEF2575_to_AR_key = "['ecFEF25-75 % ecFEV1 (%)', 'Airway resistance (%)'] -> Airway resistance (%)" 

FEV_m_list = []
HFEV1_obs_list = [2, 2.5, 3, 3.5, 4, 4.5, 5]

FEV1_obs = df_for_ID.loc[0, "ecFEV1"]
FEF2575prctFEV1_obs = df_for_ID.loc[0, "ecFEF2575%ecFEV1"]

# Query AR
for HFEV1_obs in HFEV1_obs_list: 
    # HFEV1_obs must be > ecFEV1_obs
    evidence = [[ecFEV1, FEV1_obs], [ecFEF2575prctecFEV1, FEF2575prctFEV1_obs], [HFEV1, HFEV1_obs]]
    query_res, messages = ih.infer_on_factor_graph(inf_alg, [AR], evidence, get_messages=True)

    FEV_m_list.append(messages[FEV_to_AR_key])
    FEF2575_m = messages[FEF2575_to_AR_key]


fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05, shared_xaxes=True)
for HFEV1_obs, FEV_m in zip(HFEV1_obs_list, FEV_m_list):
    ih.plot_histogram(fig, AR, FEV_m, AR.a, AR.b, 1, 1, name=f"HFEV1 = {HFEV1_obs}", annot=False)
ih.plot_histogram(fig, AR, FEF2575_m, AR.a, AR.b, 2, 1, annot=False, title=AR.name, colour="grey")
# Add message from ecFEV1/HFEV1 factor on y axis row 1 title
fig.update_yaxes(title_text="From ecFEV1/HFEV1", row=1, col=1)
fig.update_yaxes(title_text="From ecFEF2575", row=2, col=1)

# Reduce font size and margins
fig.update_layout()
# Reduce margins between plots
fig.update_layout(font=dict(size=8), margin=dict(l=10, r=10, t=10, b=10), height=250, width=600)
fig.show()